In [1]:
#This is concise version that includes "Buy_Confidence"

# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats.stats import pearsonr
import warnings
%matplotlib inline

In [2]:
# Read-in data as Excel file
vix_data_file = "C:/Users/blake/Desktop/clean_stock_data.xlsx"

In [3]:
#Convert Excel file to Dataframe
stock_data_df = pd.read_excel(vix_data_file,
sheet_name=0,
header=1,
index_col=False,
keep_default_na=False
)

In [4]:
#Create new Dataframe, set 'Date' as index, drop column named 'Date' 
time_indexed_stock_df = stock_data_df.set_index(stock_data_df['Date'])
time_indexed_stock_df = time_indexed_stock_df.drop(['Date'], axis=1)
time_indexed_stock_df.head(10)

,PX_LAST_ESA,PX_LAST_1,PX_LAST_2,PX_LAST_3,PX_LAST_4,PX_LAST_5,PX_LAST_6,PX_LAST_7,PX_LAST_8
Date,,,,,,,,,
2009-12-31,1115.10,22.95,25.80,26.00,26.30,26.35,26.30,26.40,26.35
2010-01-01,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95
2010-01-04,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95
2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.50,25.50,25.55,25.70
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.90,25.10
2010-01-08,1144.98,19.90,22.55,23.40,24.40,24.65,24.60,24.75,24.90
2010-01-11,1146.98,19.50,22.25,23.25,24.15,24.30,24.40,24.50,24.65
2010-01-12,1136.22,20.25,22.80,23.65,24.25,24.45,24.45,24.60,24.75


In [5]:
buy_conf_dict = {
    'Date':['2010-01-22', '2010-02-08', '2010-05-07', '2010-05-20', '2010-06-07',
            '2010-06-29', '2011-04-15', '2011-08-15', '2011-08-23', '2011-09-06',
            '2011-09-26', '2011-10-03', '2011-11-02', '2011-11-09', '2011-11-28',
            '2012-05-18', '2012-06-01', '2012-12-31', '2013-06-24', '2013-10-09',
            '2013-10-16', '2014-02-04', '2014-02-06', '2014-03-03', '2014-03-14',
            '2014-04-14', '2014-08-07', '2014-10-14', '2014-10-16', '2014-10-23',
            '2014-12-17', '2015-01-07', '2015-01-15', '2015-01-29', '2015-02-02',
            '2015-06-30', '2015-07-06', '2015-07-10', '2015-08-26', '2015-09-02',
            '2015-09-04', '2015-09-10', '2015-09-29', '2015-11-13', '2015-12-14',
            '2015-12-18', '2016-01-04', '2016-01-11', '2016-01-18', '2016-01-21',
            '2016-02-12', '2016-06-14', '2016-06-28', '2016-11-04', '2017-08-11',
            '2017-08-18', '2018-02-06', '2018-02-09', '2018-03-02', '2018-03-26',
            '2018-03-29', '2018-04-02', '2018-04-10', '2018-04-26', '2018-06-28',
            '2018-10-12', '2018-10-30', '2018-11-15', '2018-11-21', '2018-12-05',
            '2018-12-10', '2018-12-19', '2018-12-25', '2018-12-31', '2019-01-04',
            '2019-01-22', '2019-01-29', '2019-03-08', '2019-03-25', '2019-05-09',
            '2019-05-14', '2019-05-24', '2019-06-04', '2019-08-06', '2019-08-13',
            '2019-08-16', '2019-08-26', '2019-08-28', '2019-09-04'],
    'Buy_Conf':[8, 10, 7, 8, 7,
                9, 9, 2, 3, 3,
                2, 6, 2, 3, 4,
                5, 5, 7, 7, 8,
                5, 9, 9, 5, 7,
                5, 5, 8, 9, 2,
                8, 7, 7, 7, 7,
                5, 5, 7, 7, 8,
                7, 7, 7, 8, 7,
                7, 8, 7, 7, 7,
                7, 7, 10, 9, 7,
                7, 8, 3, 4, 2,
                5, 5, 5, 5, 5,
                3, 4, 3, 4, 2,
                2, 5, 5, 4, 4,
                5, 6, 4, 3, 7,
                7, 3, 2, 8, 7,
                7, 7, 8, 7
               ]
    }


In [6]:
buy_conf_df = pd.DataFrame(buy_conf_dict)
buy_conf_df.count()

Date        89
Buy_Conf    89
dtype: int64

In [7]:
#Create new Dataframe, set 'Date' as index, drop column named 'Date' 
conf_time_indexed_df = buy_conf_df.set_index(buy_conf_df['Date'])
conf_time_indexed_df = conf_time_indexed_df.drop(['Date'], axis=1)
conf_time_indexed_df.head(10)

,Buy_Conf
Date,
2010-01-22,8
2010-02-08,10
2010-05-07,7
2010-05-20,8
2010-06-07,7
2010-06-29,9
2011-04-15,9
2011-08-15,2
2011-08-23,3


In [8]:
#Joined Dataframes where index ('Date') matches

vix_joined_buy_conf_df = conf_time_indexed_df.join(time_indexed_stock_df)
vix_joined_buy_conf_df.head(10)
# print(vix_joined_buy_conf_df['PX_LAST_1'][0:29])
# print(vix_joined_buy_conf_df['PX_LAST_1'][30:59])
# print(vix_joined_buy_conf_df['PX_LAST_1'][60:88])
#Some of these original "Erkan Dates" are Nan
#2011-04-16, 2011-09-25, 2012-05-19, 2015-09-05, 2015-11-14, 2015-12-19, 2017-08-12
#vix_joined_buy_conf_df.isna().sum()

#Blake went back and changed dates to true 'trading day'
#New dates (respectively)
#2011-04-15, 2011-09-26, 2012-05-18, 2015-09-04, 2015-11-13, 2015-12-18, 2017-08-11

#Note: Mostly Saturday's (meaning Friday important?)

,Buy_Conf,PX_LAST_ESA,PX_LAST_1,PX_LAST_2,PX_LAST_3,PX_LAST_4,PX_LAST_5,PX_LAST_6,PX_LAST_7,PX_LAST_8
Date,,,,,,,,,,
2010-01-22,8,1091.76,24.75,25.15,25.35,25.40,25.35,25.50,25.35,25.15
2010-02-08,10,1056.74,26.40,26.20,26.30,26.20,26.30,26.35,26.25,26.35
2010-05-07,7,1110.88,32.95,30.95,30.90,30.75,31.10,31.00,30.85,30.80
2010-05-20,8,1071.59,35.95,35.70,34.00,34.30,33.80,33.30,32.90,33.15
2010-06-07,7,1050.47,35.45,34.10,33.15,33.45,33.05,32.75,31.95,32.40
2010-06-29,9,1041.24,33.50,33.45,34.75,34.75,34.25,33.50,34.20,33.80
2011-04-15,9,1319.68,16.75,19.65,21.15,22.05,22.70,23.50,24.20,24.50
2011-08-15,2,1204.49,32.10,26.70,25.95,25.45,24.50,25.70,25.80,25.95
2011-08-23,3,1162.35,34.30,31.75,30.60,28.90,29.90,29.85,29.80,30.05


In [9]:
#Drop 'Buy_Conf' and 'PX_LAST_ESA ("SPX")' columns
vix_for_conf_dates_df = vix_joined_buy_conf_df.drop(columns=['Buy_Conf', 'PX_LAST_ESA'])

#Calculate Standard Deviation of all (and only) VIX Futures on "Erkan Dates"
std_dev_vix = vix_for_conf_dates_df.std(axis=1)

In [10]:
#Add column of 1/StdDev to joined DataFrame
#Because we want "closer numbers" (lower StdDev) to have "more weight"
vix_joined_buy_conf_df.insert(loc=0, column='1/StdDev', value=1/std_dev_vix)

In [11]:
#Rename columns
vix_joined_buy_conf_df.rename(columns={
    'PX_LAST_ESA': 'SPX',
    'PX_LAST_1': 'VIX_1',
    'PX_LAST_2': 'VIX_2',
    'PX_LAST_3': 'VIX_3',
    'PX_LAST_4': 'VIX_4',
    'PX_LAST_5': 'VIX_5',
    'PX_LAST_6': 'VIX_6',
    'PX_LAST_7': 'VIX_7',
    'PX_LAST_8': 'VIX_8'
})

,1/StdDev,Buy_Conf,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8
Date,,,,,,,,,,,
2010-01-22,4.264014,8,1091.76,24.750,25.150,25.350,25.400,25.350,25.500,25.350,25.150
2010-02-08,13.719887,10,1056.74,26.400,26.200,26.300,26.200,26.300,26.350,26.250,26.350
2010-05-07,1.368314,7,1110.88,32.950,30.950,30.900,30.750,31.100,31.000,30.850,30.800
2010-05-20,0.877359,8,1071.59,35.950,35.700,34.000,34.300,33.800,33.300,32.900,33.150
2010-06-07,0.918146,7,1050.47,35.450,34.100,33.150,33.450,33.050,32.750,31.950,32.400
2010-06-29,1.841149,9,1041.24,33.500,33.450,34.750,34.750,34.250,33.500,34.200,33.800
2011-04-15,0.384458,9,1319.68,16.750,19.650,21.150,22.050,22.700,23.500,24.200,24.500
2011-08-15,0.427878,2,1204.49,32.100,26.700,25.950,25.450,24.500,25.700,25.800,25.950
2011-08-23,0.593358,3,1162.35,34.300,31.750,30.600,28.900,29.900,29.850,29.800,30.050


In [12]:
vix_joined_buy_conf_df.describe()

,1/StdDev,Buy_Conf,PX_LAST_ESA,PX_LAST_1,PX_LAST_2,PX_LAST_3,PX_LAST_4,PX_LAST_5,PX_LAST_6,PX_LAST_7,PX_LAST_8
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,2.232762,5.898876,2079.124494,22.424101,21.641292,21.493820,21.469663,21.517697,21.625562,21.731180,21.836798
std,2.203106,2.137569,587.695192,6.144478,5.405638,5.081045,4.953781,4.872683,4.758185,4.688347,4.640124
min,0.280068,2.000000,1041.240000,14.975000,15.125000,15.375000,15.325000,15.725000,16.525000,16.925000,17.250000
25%,1.107698,4.000000,1773.430000,18.125000,18.275000,18.150000,18.300000,18.150000,18.475000,18.675000,18.900000
50%,1.546407,7.000000,2021.250000,20.175000,19.650000,19.600000,19.775000,19.850000,19.875000,19.950000,20.050000
75%,2.791291,7.000000,2649.930000,24.750000,22.675000,22.225000,22.325000,22.675000,22.775000,22.775000,22.825000
max,13.719887,10.000000,2937.780000,45.050000,40.550000,36.700000,37.050000,35.600000,35.150000,35.300000,34.650000
